In [3]:
from pandas import read_csv as pd_read_csv


disease_data = pd_read_csv('dataset.csv')
symptom_desc = pd_read_csv('symptom_Description.csv')
symptom_prec = pd_read_csv('symptom_precaution.csv')


symptom_desc.columns = [col.strip() for col in symptom_desc.columns]
symptom_prec.columns = [col.strip() for col in symptom_prec.columns]


symptom_desc_dict = symptom_desc.set_index('Disease').to_dict()['Description']
symptom_prec_dict = symptom_prec.set_index('Disease').to_dict()

def write_rules(file_path, disease_data, symptom_desc_dict, symptom_prec_dict):
    import pandas as pd  # Importing pandas with the 'pd' alias in the function scope

    with open(file_path, 'w') as file:
        # Define templates
        file.write('''
;; Define templates
(deftemplate illness
    (multislot symptoms)  ;; Use multislot for multiple values
    (slot description)
    (multislot precautions)
    (slot name))

;; Define symptom facts
''')

       
        for index, row in disease_data.iterrows():
            illness_name = row['Disease'].replace(' ', '_').replace('(', '').replace(')', '')
            # Gather all symptom columns into a list
            symptoms = [row[s].strip() for s in disease_data.columns if 'Symptom' in s and not pd.isnull(row[s])]
            description = symptom_desc_dict.get(row['Disease'], 'No description available')
            
            precautions = [row[p].strip() for p in disease_data.columns if 'Precaution' in p and not pd.isnull(row[p])]
            precautions_str = ' '.join(f'"{p}"' for p in precautions)
            
            file.write(f'''
(defrule {illness_name}
    (or {" ".join(f'(symptom (name "{symptom}"))' for symptom in symptoms)})
    =>
    (assert (illness (name "{row['Disease']}") 
                     (symptoms {" ".join(f'"{symptom}"' for symptom in symptoms)}) 
                     (description "{description}") 
                     (precautions {precautions_str})))
)
''')


write_rules('medical_rules.clp', disease_data, symptom_desc_dict, symptom_prec_dict)
